In [1]:
from bs4 import BeautifulSoup as bs

In [18]:
import urllib
import feedparser
import pickle

In [ ]:
## querying the nyt data

https://api.nytimes.com/svc/books/v3/lists/overview.json?api-key=<your-api-key>
http://api.nytimes.com/svc/search/v1/article?format=json&query=smoking&api-key=####

In [96]:
def getArticles(date, api_key, json_file_path):
    # LOOP THROUGH THE 101 PAGES NYTIMES ALLOWS FOR THAT DATE
    for page in range(101):
        try:
            request_string = "http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=" + date + "&end_date=" + date + "&page=" + str(page) + "&api-key=" + api_key
            response = urllib.urlopen(request_string)
            content = response.read()
            if content:
                articles = convert(json.loads(content))
                # if there are articles here
                if len(articles["response"]["docs"]) >= 1:
                    json_file_name = getJsonFileName(date, page, json_file_path)
                    json_file = open(json_file_name, 'w')
                    json_file.write(content)
                    json_file.close()
                # if no more articles, go to next date
                else:
                    return
            # else:
            #     break
            time.sleep(3)
        except HTTPError as e:
            logging.error("HTTPError on page %s on %s (err no. %s: %s) Here's the URL of the call: %s", page, date, e.code, e.reason, request_string)
        except: 
            logging.error("Error on %s page %s: %s", date, file_number, sys.exc_info()[0])
            continue

In [45]:
import urllib
import json
import datetime
import time
import sys
import argparse
import logging


# helper function to iterate through dates
def daterange( start_date, end_date ):
    if start_date <= end_date:
        for n in range( ( end_date - start_date ).days + 1 ):
            yield start_date + datetime.timedelta( n )
    else:
        for n in range( ( start_date - end_date ).days + 1 ):
            yield start_date - datetime.timedelta( n )
 
# helper function to get json into a form I can work with       
def convert(input):
    if isinstance(input, dict):
        return {convert(key): convert(value) for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [convert(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input
 
# helpful function to figure out what to name individual JSON files        
def getJsonFileName(date, page, json_file_path):
    json_file_name = ".".join([date,str(page),'json'])
    json_file_name = "".join([json_file_path,json_file_name])
    return json_file_name
 
# helpful function for processing keywords, mostly    
def getMultiples(items, key):
    values_list = ""
    if len(items) > 0:
        num_keys = 0
        for item in items:
            if num_keys == 0:
                values_list = item[key]                
            else:
                values_list =  "; ".join([values_list,item[key]])
            num_keys += 1
    return values_list
 
# get the articles from the NYTimes Article API

def getArticles(date, api_key, json_file_path):
    # LOOP THROUGH THE 101 PAGES NYTIMES ALLOWS FOR THAT DATE
    for page in range(101):
        try:
            request_string = "http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=" + date + "&end_date=" + date + "&page=" + str(page) + "&api-key=" + api_key
            response = urllib.request.urlopen(request_string)
            content = response.read()
            if content:
                articles = convert(json.loads(content))
                # if there are articles here
                if len(articles["response"]["docs"]) >= 1:
                    json_file_name = getJsonFileName(date, page, json_file_path)
                    json_file = open(json_file_name, 'w')
                    json_file.write(content)
                    json_file.close()
                # if no more articles, go to next date
                else:
                    return
            # else:
            #     break
            time.sleep(3)
        except HTTPError as e:
            logging.error("HTTPError on page %s on %s (err no. %s: %s) Here's the URL of the call: %s", page, date, e.code, e.reason, request_string)
        except: 
            logging.error("Error on %s page %s: %s", date, file_number, sys.exc_info()[0])
            continue
 
# parse the JSON files you stored into a tab-delimited file
def parseArticles(date, csv_file_name, json_file_path):
    for file_number in range(101):
        # get the articles and put them into a dictionary
        try:
            file_name = getJsonFileName(date,file_number, json_file_path)
            in_file = open(file_name, 'r')
            articles = convert(json.loads(in_file.read()))
            in_file.close()
        except IOError as e:
            logging.error("IOError in %s page %s: %s %s", date, file_number, e.errno, e.strerror)
            continue
 
        # if there are articles in that document, parse them
        if len(articles["response"]["docs"]) >= 1:  
            # open the CSV for appending
            try:
                out_file = open(csv_file_name, 'ab')
            except IOError as e:
                logging.error("IOError: %s %s", date, file_number, e.errno, e.strerror)
                continue
 
            # loop through the articles putting what we need in a CSV   
            try:
                for article in articles["response"]["docs"]:
                    # if (article["source"] == "The New York Times" and article["document_type"] == "article"):
                    keywords = ""
                    keywords = getMultiples(article["keywords"],"value")
 
                    # should probably pull these if/else checks into a module
                    variables = [
                        article["pub_date"], 
                        keywords, 
                        str(article["headline"]["main"]).decode("utf8").replace("\n","") if "main" in article["headline"].keys() else "", 
                        str(article["source"]).decode("utf8") if "source" in article.keys() else "", 
                        str(article["document_type"]).decode("utf8") if "document_type" in article.keys() else "", 
                        article["web_url"] if "web_url" in article.keys() else "",
                        str(article["news_desk"]).decode("utf8") if "news_desk" in article.keys() else "",
                        str(article["section_name"]).decode("utf8") if "section_name" in article.keys() else "",
                        str(article["snippet"]).decode("utf8").replace("\n","") if "snippet" in article.keys() else "",
                        str(article["lead_paragraph"]).decode("utf8").replace("\n","") if "lead_paragraph" in article.keys() else "",
                        ]
                    line = "\t".join(variables)
                    out_file.write(line.encode("utf8")+"\n")
            except KeyError as e:
                logging.error("KeyError in %s page %s: %s %s", date, file_number, e.errno, e.strerror)
                continue
            except (KeyboardInterrupt, SystemExit):
                raise
            except: 
                logging.error("Error on %s page %s: %s", date, file_number, sys.exc_info()[0])
                continue
 
            out_file.close()
        else:
            break

In [42]:
import urllib
import json
import datetime
import time
import sys
import argparse
import logging
 
# helper function to iterate through dates
def daterange( start_date, end_date ):
    if start_date <= end_date:
        for n in range( ( end_date - start_date ).days + 1 ):
            yield start_date + datetime.timedelta( n )
    else:
        for n in range( ( start_date - end_date ).days + 1 ):
            yield start_date - datetime.timedelta( n )
 
# helper function to get json into a form I can work with       
def convert(input):
    if isinstance(input, dict):
        return {convert(key): convert(value) for key, value in input.iteritems()}
    elif isinstance(input, list):
        return [convert(element) for element in input]
    elif isinstance(input, unicode):
        return input.encode('utf-8')
    else:
        return input
 
# helpful function to figure out what to name individual JSON files        
def getJsonFileName(date, page, json_file_path):
    json_file_name = ".".join([date,str(page),'json'])
    json_file_name = "".join([json_file_path,json_file_name])
    return json_file_name
 
# helpful function for processing keywords, mostly    
def getMultiples(items, key):
    values_list = ""
    if len(items) > 0:
        num_keys = 0
        for item in items:
            if num_keys == 0:
                values_list = item[key]                
            else:
                values_list =  "; ".join([values_list,item[key]])
            num_keys += 1
    return values_list
 
# get the articles from the NYTimes Article API    
def getArticles(date, api_key, json_file_path):
    # LOOP THROUGH THE 101 PAGES NYTIMES ALLOWS FOR THAT DATE
    for page in range(101):
        try:
            request_string = "http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=" + date + "&end_date=" + date + "&page=" + str(page) + "&api-key=" + api_key
            response = urllib.urlopen(request_string)
            content = response.read()
            if content:
                
                articles = convert(json.loads(content))
                # if there are articles here
                if len(articles["response"]["docs"]) >= 1:
                    json_file_name = getJsonFileName(date, page, json_file_path)
                    json_file = open(json_file_name, 'w')
                    json_file.write(content)
                    json_file.close()
                # if no more articles, go to next date
                else:
                    return
            # else:
            #     break
            time.sleep(3)
        except HTTPError as e:
            logging.error("HTTPError on page %s on %s (err no. %s: %s) Here's the URL of the call: %s", page, date, e.code, e.reason, request_string)
        except: 
            logging.error("Error on %s page %s: %s", date, file_number, sys.exc_info()[0])
            continue
 
# parse the JSON files you stored into a tab-delimited file
def parseArticles(date, csv_file_name, json_file_path):
    for file_number in range(101):
        # get the articles and put them into a dictionary
        try:
            file_name = getJsonFileName(date,file_number, json_file_path)
            in_file = open(file_name, 'r')
            articles = convert(json.loads(in_file.read()))
            in_file.close()
        except IOError as e:
            logging.error("IOError in %s page %s: %s %s", date, file_number, e.errno, e.strerror)
            continue
 
        # if there are articles in that document, parse them
        if len(articles["response"]["docs"]) >= 1:  
            # open the CSV for appending
            try:
                out_file = open(csv_file_name, 'ab')
            except IOError as e:
                logging.error("IOError: %s %s", date, file_number, e.errno, e.strerror)
                continue
 
            # loop through the articles putting what we need in a CSV   
            try:
                for article in articles["response"]["docs"]:
                    # if (article["source"] == "The New York Times" and article["document_type"] == "article"):
                    keywords = ""
                    keywords = getMultiples(article["keywords"],"value")
 
                    # should probably pull these if/else checks into a module
                    variables = [
                        article["pub_date"], 
                        keywords, 
                        str(article["headline"]["main"]).decode("utf8").replace("\n","") if "main" in article["headline"].keys() else "", 
                        str(article["source"]).decode("utf8") if "source" in article.keys() else "", 
                        str(article["document_type"]).decode("utf8") if "document_type" in article.keys() else "", 
                        article["web_url"] if "web_url" in article.keys() else "",
                        str(article["news_desk"]).decode("utf8") if "news_desk" in article.keys() else "",
                        str(article["section_name"]).decode("utf8") if "section_name" in article.keys() else "",
                        str(article["snippet"]).decode("utf8").replace("\n","") if "snippet" in article.keys() else "",
                        str(article["lead_paragraph"]).decode("utf8").replace("\n","") if "lead_paragraph" in article.keys() else "",
                        ]
                    line = "\t".join(variables)
                    out_file.write(line.encode("utf8")+"\n")
            except KeyError as e:
                logging.error("KeyError in %s page %s: %s %s", date, file_number, e.errno, e.strerror)
                continue
            except (KeyboardInterrupt, SystemExit):
                raise
            except: 
                logging.error("Error on %s page %s: %s", date, file_number, sys.exc_info()[0])
                continue
 
            out_file.close()
        else:
            break
 
# Main function where stuff gets done
 
def main():
    parser = argparse.ArgumentParser(description="A Python tool for grabbing data from the New York Times Article API.")
    parser.add_argument('-j','--json', required=True, help="path to the folder where you want the JSON files stored")
    parser.add_argument('-c','--csv', required=True, help="path to the file where you want the CSV file stored")
    parser.add_argument('-k','--key', required=True, help="your NY Times Article API key")
    # parser.add_argument('-s','--start-date', required=True, help="start date for collecting articles")
    # parser.add_argument('-e','--end-date', required=True, help="end date for collecting articles")
    args = parser.parse_args()
 
    json_file_path = args.json
    csv_file_name = args.csv
    api_key = args.key    
    start = datetime.date( year = 2013, month = 1, day = 1 )
    end = datetime.date( year = 2013, month = 1, day = 1 )
    log_file = "".join([json_file_path,"getTimesArticles_testing.log"])
    logging.basicConfig(filename=log_file, level=logging.INFO)
 
    logging.info("Getting started.") 
    try:
        # LOOP THROUGH THE SPECIFIED DATES
        for date in daterange( start, end ):
            date = date.strftime("%Y%m%d")
            logging.info("Working on %s." % date)
            getArticles(date, api_key, json_file_path)
            parseArticles(date, csv_file_name, json_file_path)
    except:
        logging.error("Unexpected error: %s", str(sys.exc_info()[0]))
    finally:
        logging.info("Finished.")
 
if __name__ == '__main__' :
    main()

In [ ]:
getArticles(date, api_key, json_file_path)

In [89]:
## the guy way of getting the papers

In [146]:
import urllib
import json
import datetime
import time
import sys
import argparse
import logging
from urllib.error import HTTPError
 
# helper function to iterate through dates
def daterange( start_date, end_date ):
    if start_date <= end_date:
        for n in range( ( end_date - start_date ).days + 1 ):
            yield start_date + datetime.timedelta( n )
    else:
        for n in range( ( start_date - end_date ).days + 1 ):
            yield start_date - datetime.timedelta( n )
 
# helper function to get json into a form I can work with       
def convert(input):
    if isinstance(input, dict):
        return {convert(key): convert(value) for key, value in input.items()}
    elif isinstance(input, list):
        return [convert(element) for element in input]
    elif isinstance(input, str):
        return input.encode('utf-8')
    else:
        return input
 
# helpful function to figure out what to name individual JSON files        
def getJsonFileName(date, page, json_file_path):
    json_file_name = ".".join([date,str(page),'json'])
    json_file_name = "".join([json_file_path,json_file_name])
    return json_file_name
 
# helpful function for processing keywords, mostly    
def getMultiples(items, key):
    values_list = ""
    if len(items) > 0:
        num_keys = 0
        for item in items:
            if num_keys == 0:
                values_list = item[key]                
            else:
                values_list =  "; ".join([values_list,item[key]])
            num_keys += 1
    return values_list
 
# get the articles from the NYTimes Article API    
def getArticles(date, api_key, json_file_path):
    # LOOP THROUGH THE 101 PAGES NYTIMES ALLOWS FOR THAT DATE
    for page in range(101):
        try:
            request_string = "http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=" + date + "&end_date=" + date + "&page=" + str(page) + "&api-key=" + api_key
            print(request_string)
            
            response = urllib.request.urlopen(request_string)
            content = response.read()
            
            if content:
                print("there is content")
                articles = convert(json.loads(content))
                # if there are articles here
                if len(articles["response"]["docs"]) >= 1:
                    print("getting the paper")
                    json_file_name = getJsonFileName(date, page, json_file_path)
                    json_file = open(json_file_name, 'w')
                    json_file.write(content)
                    json_file.close()
                # if no more articles, go to next date
                else:
                    return
            else:
                time.sleep(3)
        except HTTPError as e:
            print("error in http ")
            logging.error("HTTPError on page %s on %s (err no. %s: %s) Here's the URL of the call: %s", page, date, e.code, e.reason, request_string)
        except:
            print("error in date")
            logging.error("Error on %s page %s: %s", date, file_number, sys.exc_info()[0])
            continue
 
# parse the JSON files you stored into a tab-delimited file
def parseArticles(date, csv_file_name, json_file_path):
    for file_number in range(101):
        # get the articles and put them into a dictionary
        try:
            file_name = getJsonFileName(date,file_number, json_file_path)
            in_file = open(file_name, 'r')
            articles = convert(json.loads(in_file.read()))
            in_file.close()
        except IOError as e:
            logging.error("IOError in %s page %s: %s %s", date, file_number, e.errno, e.strerror)
            continue
 
        # if there are articles in that document, parse them
        if len(articles["response"]["docs"]) >= 1:  
            # open the CSV for appending
            try:
                out_file = open(csv_file_name, 'ab')
            except IOError as e:
                logging.error("IOError: %s %s", date, file_number, e.errno, e.strerror)
                continue
 
            # loop through the articles putting what we need in a CSV   
            try:
                for article in articles["response"]["docs"]:
                    # if (article["source"] == "The New York Times" and article["document_type"] == "article"):
                    keywords = ""
                    keywords = getMultiples(article["keywords"],"value")
 
                    # should probably pull these if/else checks into a module
                    variables = [
                        article["pub_date"], 
                        keywords, 
                        str(article["headline"]["main"]).decode("utf8").replace("\n","") if "main" in article["headline"].keys() else "", 
                        str(article["source"]).decode("utf8") if "source" in article.keys() else "", 
                        str(article["document_type"]).decode("utf8") if "document_type" in article.keys() else "", 
                        article["web_url"] if "web_url" in article.keys() else "",
                        str(article["news_desk"]).decode("utf8") if "news_desk" in article.keys() else "",
                        str(article["section_name"]).decode("utf8") if "section_name" in article.keys() else "",
                        str(article["snippet"]).decode("utf8").replace("\n","") if "snippet" in article.keys() else "",
                        str(article["lead_paragraph"]).decode("utf8").replace("\n","") if "lead_paragraph" in article.keys() else "",
                        ]
                    line = "\t".join(variables)
                    out_file.write(line.encode("utf8")+"\n")
            except KeyError as e:
                logging.error("KeyError in %s page %s: %s %s", date, file_number, e.errno, e.strerror)
                continue
            except (KeyboardInterrupt, SystemExit):
                raise
            except: 
                logging.error("Error on %s page %s: %s", date, file_number, sys.exc_info()[0])
                continue
 
            out_file.close()
        else:
            break
 
# Main function where stuff gets done
json_file_path = "test.json"
csv_file_name = "test.csv"    
start = datetime.date( year = 2013, month = 1, day = 1 )
end = datetime.date( year = 2013, month = 1, day = 1 )

log_file = "".join([json_file_path,"getTimesArticles_testing.log"])
logging.basicConfig(filename=log_file, level=logging.INFO)
 
logging.info("Getting started.") 
try:
    # LOOP THROUGH THE SPECIFIED DATES
    for date in daterange( start, end ):
        print("here")
        date = date.strftime("%Y%m%d")
        logging.info("Working on %s." % date)
        getArticles(date, api_key, json_file_path)
        parseArticles(date, csv_file_name, json_file_path)
except:
    logging.error("Unexpected error: %s", str(sys.exc_info()[0]))
finally:
    logging.info("Finished.")

here
http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=20130101&end_date=20130101&page=0&api-key=923f498f9f764c228a68501cf957355b
there is content
error in date


In [126]:
start = datetime.date( year = 2013, month = 1, day = 1 )
end = datetime.date( year = 2013, month = 1, day = 1 )

In [128]:
date = start.strftime("%Y%m%d")

In [147]:
date = "20130101"
getArticles(date, api_key, "test.json")

http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=20130101&end_date=20130101&page=0&api-key=923f498f9f764c228a68501cf957355b
there is content
error in date


NameError: name 'file_number' is not defined

In [2]:
# the newest way

In [ ]:
'''step 1: input query information'''
apiUrl='http://api.nytimes.com/svc/search/v2/articlesearch.json?q=occupy+wall+street'
apiDate='begin_date=20110901&end_date=20120214'             # set the date here
fields='fields=body%2Curl%2Ctitle%2Cdate%2Cdes_facet%2Cdesk_facet%2Cbyline'
offset='offset=0'
key='api-key='+api_key  # input your key here



In [ ]:
'''step 2: get the number of offset/pages'''
link=[apiUrl, apiDate, fields, offset, key]
ReqUrl='&'.join(link)
print(ReqUrl)
jstr = urllib.request.urlopen(ReqUrl).read()  # t = jstr.strip('()')
ts = json.loads( jstr )
#print(ts)
number=10 #  the number of queries, maximal is 10, you can not change it
print(number) 
seq=range(number)  # this is not a good way
#print(seq) 

'''step 3: crawl the data and dump into csv'''
import csv
addressForSavingData= "nyt.csv"
file = open(addressForSavingData,'wb') # save to csv file
for i in seq:
    nums=str(i)
    offsets=''.join(['offset=', nums]) # I made error here, and print is a good way to test
    links=[apiUrl, query, apiDate, fields, offsets, key]
    ReqUrls='&'.join(links)
    print("*_____________*", ReqUrls) 
    jstrs = urllib.request.urlopen(ReqUrl).read()
    t = jstrs.strip('()')
    tss= json.loads( t )  # error no joson object
    result = tss['results']
    for ob in result:
        title=ob['title']  # body=ob['body']   # body,url,title,date,des_facet,desk_facet,byline
        print(title) 
        url=ob['url']
        date=ob['date'] # desk_facet=ob['desk_facet']  # byline=ob['byline'] # some author names don't exist
        w = csv.writer(file,delimiter=',',quotechar='|', quoting=csv.QUOTE_MINIMAL)
        w.writerow((date, title, url)) # write it out
file.close()
pass



In [47]:
##### OK BACK TO MY WAY ---- SKETCH OF THE MODULE

request_string = "http://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=" + date + "&end_date=" + date + "&page=" + str(page) + "&api-key=" + api_key

In [58]:
def NYTquery(begin_date, end_date, page=1, api_key = '923f498f9f764c228a68501cf957355b'):
    # initial settings, will be part of class variable:
    base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date='
    
    # function own variables, needed for the query:
    
    #search_query
    search_query = str(begin_date) + "&end_date=" + str(end_date) + "&page=" + str(page) + "&api-key=" + api_key
    
    # actual NYT query
    response = urllib.request.urlopen(base_url+search_query).read()
    
    # parse the response using feedparser
    return response

In [48]:
base_url = 'https://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date='

In [50]:
search_query = str(20161201) + "&end_date=" + str(20161231) + "&page=" + str(1) + "&api-key=" + api_key

In [61]:
data = NYTquery(20161201,20161231)

In [71]:
data

b'{"response":{"meta":{"hits":20942,"time":45,"offset":10},"docs":[{"web_url":"http:\\/\\/www.nytimes.com\\/reuters\\/2016\\/12\\/31\\/world\\/europe\\/31reuters-mideast-crisis-syria-blast.html","snippet":"Two suicide bombers detonated their explosives, killing at least two security officers in the Syrian coastal city of Tartous, Syrian state television reported after midnight on Sunday....","lead_paragraph":"Two suicide bombers detonated their explosives, killing at least two security officers in the Syrian coastal city of Tartous, Syrian state television reported after midnight on Sunday.","abstract":null,"print_page":null,"blog":[],"source":"Reuters","multimedia":[],"headline":{"main":"Two Suicide Attackers Kill at Least Two in Syria\'s Tartous: State TV, Monitors","print_headline":"Two Suicide Attackers Kill at Least Two in Syria\'s Tartous: State TV, Monitors"},"keywords":[],"pub_date":"2016-12-31T23:27:28+0000","document_type":"article","news_desk":null,"section_name":"World","su

In [80]:
content = data

In [138]:
# helper function to get json into a form I can work with       
def convert(input):
    if isinstance(input, dict):
        return {convert(key): convert(value) for key, value in input.items()}
    elif isinstance(input, list):
        return [convert(element) for element in input]
    elif isinstance(input, str):
        return input.encode('utf-8')
    else:
        return input

In [139]:
articles = convert(json.loads(content))

In [140]:
articles

{b'copyright': b'Copyright (c) 2013 The New York Times Company.  All Rights Reserved.',
 b'response': {b'docs': [{b'_id': b'58683eea95d0e03926078817',
    b'abstract': None,
    b'blog': [],
    b'byline': {b'organization': b'REUTERS',
     b'original': b'By REUTERS',
     b'person': []},
    b'document_type': b'article',
    b'headline': {b'main': b"Two Suicide Attackers Kill at Least Two in Syria's Tartous: State TV, Monitors",
     b'print_headline': b"Two Suicide Attackers Kill at Least Two in Syria's Tartous: State TV, Monitors"},
    b'keywords': [],
    b'lead_paragraph': b'Two suicide bombers detonated their explosives, killing at least two security officers in the Syrian coastal city of Tartous, Syrian state television reported after midnight on Sunday.',
    b'multimedia': [],
    b'news_desk': None,
    b'print_page': None,
    b'pub_date': b'2016-12-31T23:27:28+0000',
    b'section_name': b'World',
    b'slideshow_credits': None,
    b'snippet': b'Two suicide bombers detona

In [134]:
if content:
    articles = convert(json.loads(content))
    # if there are articles here
    if len(articles["response"]["docs"]) >= 1:
        #json_file_name = getJsonFileName(date, page, json_file_path)
        json_file = open('tester.json', 'w')
        json_file.write(content)
        json_file.close()

NameError: name 'unicode' is not defined

In [84]:
kk = json.loads(content)

In [86]:
kk

{'copyright': 'Copyright (c) 2013 The New York Times Company.  All Rights Reserved.',
 'response': {'docs': [{'_id': '58683eea95d0e03926078817',
    'abstract': None,
    'blog': [],
    'byline': {'organization': 'REUTERS',
     'original': 'By REUTERS',
     'person': []},
    'document_type': 'article',
    'headline': {'main': "Two Suicide Attackers Kill at Least Two in Syria's Tartous: State TV, Monitors",
     'print_headline': "Two Suicide Attackers Kill at Least Two in Syria's Tartous: State TV, Monitors"},
    'keywords': [],
    'lead_paragraph': 'Two suicide bombers detonated their explosives, killing at least two security officers in the Syrian coastal city of Tartous, Syrian state television reported after midnight on Sunday.',
    'multimedia': [],
    'news_desk': None,
    'print_page': None,
    'pub_date': '2016-12-31T23:27:28+0000',
    'section_name': 'World',
    'slideshow_credits': None,
    'snippet': 'Two suicide bombers detonated their explosives, killing at l

In [66]:
import pandas as pd

In [57]:
base_url+search_query

'https://api.nytimes.com/svc/search/v2/articlesearch.json?begin_date=20161201&end_date=20161231&page=1&api-key=923f498f9f764c228a68501cf957355b'

In [ ]:
## ok now turn the json to csv.

In [60]:
import json
import csv

In [68]:
df = pd.read_json(data)

In [72]:
csv = df.to_csv()

In [76]:
## flattening the json

In [77]:
def flattenjson( b, delim ):
    val = {}
    for i in b.keys():
        if isinstance( b[i], dict ):
            get = flattenjson( b[i], delim )
            for j in get.keys():
                val[ i + delim + j ] = get[j]
        else:
            val[i] = b[i]

    return val

In [79]:
type(data)

bytes